In [112]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch import tensor, manual_seed, optim

In [68]:
data = pd.read_csv("Activity_recog_train.csv")

In [69]:
data.head(5)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [70]:
data.tail(5)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819,30,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053,30,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811,30,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339,30,WALKING_UPSTAIRS
7351,0.351503,-0.012423,-0.203867,-0.269270,-0.087212,0.177404,-0.377404,-0.038678,0.229430,0.269013,...,-0.740738,-0.280088,-0.007739,-0.056088,-0.616956,-0.783267,0.246809,0.036695,30,WALKING_UPSTAIRS


In [71]:
data.describe(include="all")

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LAYING
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1407
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515,17.413085,NaN
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122,8.975143,NaN
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,NaN
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414,8.000000,NaN
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181,19.000000,NaN
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659,26.000000,NaN


In [72]:
Dtypes = []
for col in data.columns:
    Dtypes.append(str(data[col].dtype))

print(f"Total columns ===> {len(data.columns)}")
count = Counter(Dtypes)
print(dict(count))

Total columns ===> 563
{'float64': 561, 'int64': 1, 'object': 1}


In [73]:
Classes = list(set(data["Activity"]))
Classes

['SITTING',
 'WALKING',
 'WALKING_UPSTAIRS',
 'WALKING_DOWNSTAIRS',
 'LAYING',
 'STANDING']

In [74]:
encoder = LabelEncoder()
data["Activity"] = encoder.fit_transform(data["Activity"])

In [75]:
Features = data.drop(["Activity"], axis=1)
Target = data["Activity"]

Train Test Split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size=0.2, random_state=40)

In [84]:
X_train = tensor(X_train.values)
X_test = tensor(X_test.values)
y_train = tensor(y_train.values)
y_test = tensor(y_test.values)

In [85]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

torch.Size([5881, 562])
torch.Size([1471, 562])
torch.Size([5881])
torch.Size([1471])


In [87]:
TrainData = TensorDataset(X_train, y_train)
TestData = TensorDataset(X_test, y_test)

In [90]:
batch_size = 56
TrainLoader = DataLoader(TrainData, batch_size=batch_size, shuffle=True)
TestLoader = DataLoader(TestData, batch_size=batch_size, shuffle=True)

Constructing a Neural Network

In [107]:
class Network(nn.Module):
    def __init__(self, inp_size, out_size, layers, DropOutRatio = 0.5):
        super().__init__()
        self.layer_list = []
        self.layer_list.append(nn.Dropout(DropOutRatio))
        self.layer_list.append(nn.BatchNorm1d(inp_size))

        for layer_size in layers:
            self.layer_list.append(nn.Linear(inp_size, layer_size))
            self.layer_list.append(nn.ReLU())
            self.layer_list.append(nn.BatchNorm1d(layer_size))
            self.layer_list.append(nn.Dropout(DropOutRatio))
            inp_size = layer_size

        self.layer_list.append(nn.Linear(inp_size, out_size))
        self.layer_list.append(nn.Softmax(dim=1))

        self.Layers = nn.Sequential(*self.layer_list)
    
    def forward(self, X):
        output = self.Layers(X)
        return output

In [110]:
manual_seed(100)
Net = Network(len(X_train[1]), len(y_train.unique()), [50, 30], DropOutRatio=0.4)
Net

Network(
  (Layers): Sequential(
    (0): Dropout(p=0.4, inplace=False)
    (1): BatchNorm1d(562, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=562, out_features=50, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=50, out_features=30, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=30, out_features=6, bias=True)
    (11): Softmax(dim=1)
  )
)

In [113]:
# Defining loss and optimizer function
loss_function = nn.BCELoss()
optimizer = optim.SGD(Net.parameters(), lr=0.001, momentum=0.9)

In [114]:
Epochs = 1
Loss = []
Accuracy = []

for epoch in range(Epochs):
    
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    for batchNumber, data in enumerate(TrainLoader, 0):

        optimizer.zero_grad()
        y_pred = Net(data[0])
        print(y_pred)
        print(data[1])

RuntimeError: mixed dtype (CPU): expect input to have scalar type of BFloat16

In [117]:
for batchNumber, data in enumerate(TrainLoader, 0):
    print(data[0].dtype)
    print(data[1].dtype)

torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float64
torch.int64
torch.float6